# Contents

* [<font size=4>Abstract</font>](#1)
* [<font size=4>Objective</font>](#1)
* [<font size=4>The dataset</font>](#1)
* [<font size=4>Performance Metric</font>](#1)

* [<font size=4>EDA</font>](#2)
  
    
* [<font size=4>Modeling</font>](#3)
    * [Train/Val split](#3.1)
    * [Naive approach](#3.2)
    * [Moving average](#3.3)
    * [Exponential smoothing](#3.4)
    * [ARIMA](#3.5)
    * [Prophet](#3.6)
    * [Loss for each model](#3.7)



# Abstract

Walmart, Whole Foods, Amazon, and other large department stores maintain millions of products and record millions of transactions every day. In order to optimize the profit, store manager must keep balance between inventory, current demand, and they are relying on accurate sales prediction to make informed decisions. Most of the existing sales predictions depends on statistical inference from trends. previous forecasting methods requires a lot of extra information from customers to product detailed information and requires customer and product attribute analysis. In this project we will try to build simplified model to predict the unit sales based historical sales record. Our data is from Kaggle M5 Competition, we will use combination of statistical a machine learning including Deep Neural Networks to solve this exciting case study.


# Objective

The objective of the project is to precisely forecast the unit sales for Walmart USA. Incorrect or misleading forecasts on product sales could trigger opportunity and revenue loss for Walmart. For instance, if analyst team failed to estimate the demand for different product for different categories at different stores located in various states during long weekends or thanksgivings, they may lose opportunity to attract thousands of customers and potentially lose millions of dollars of revenue.  We will build a model that will assist the business analyst and manager to improve planning the inventory distribution, inventory storage solutions, promotions and offer decisions.


# The Dataset Information

The dataset consists of five .csv files.

1. calendar.csv - Contains the dates on which products are sold. The dates are in a yyyy/dd/mm format.

2. sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913].

3. submission.csv - Demonstrates the correct format for submission to the competition.

4. sell_prices.csv - Contains information about the price of the products sold per store and date.

5. sales_train_evaluation.csv - Available one month before the competition deadline. It will include sales for [d_1 - d_1941].

* In this competition, we need to forecast the sales for [d_1942 - d_1969]. These rows form the evaluation set. The rows [d_1914 - d_1941] form the validation set, and the remaining rows form the training set. Now, since we understand the dataset and know what to predict, let us visualize the dataset.

* Dataset was provided by Walmart USA, involves unit sales of various product sold in the USA, organized in the form of grouped time series.
*	Dataset contains unique 3049 products, classified in 3 product categories (Hobbies, Food, and Household), and 7 product departments, products are sold across 10 stores in 3 different states (CA, TX, WI )
*	In this respect, the bottom-level of the hierarchy, i.e., product-store unit sales can be mapped across either product categories or geographical regions, as follows:
* Every product has selling history of 1941 days, (test data of h=28 days not included)





![Imgur](https://i.imgur.com/WB6hm1l.png)

# Perfromance Metric

The accuracy of the point forecasts will be evaluated using the R**oot Mean Squared Scaled Error (RMSSE)**, The measure is calculated for each series as follows.

> $RMSSE =\sqrt{\frac{1}{h} * \frac{\sum_{t=n+1}^{t=n+h} {(Y_t - \hat{Y_t}})^2}{\frac{1}{n-1}*(\sum_{t=2}^{t=n}(y_t-y_{t-1)})^2}}$

where Yt  is the actual future value of the examined time series at point t,  the generated forecast, n the length of the training sample (number of historical observations), and h the forecasting horizon. After estimating RMSSE for all the time series of the competition, participant method will be ranked using** Weighted RMSSE**.

> $WRMSSE =\sum_{i=1}^{i=42840} {w_i * RMSSE}$


where W_i is the weight of the ith series of the competition?  n is the total number of time series in competition, A lower WRMSSE score is better. *weight of each series will be computed based on the last 28 observations of the training sample of the dataset*, i.e., **the cumulative actual dollar sales that each series displayed in that period (sum of units sold multiplied by their respective price**).


### Import Library

In [1]:

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import os
import gc
import time
import math
import datetime

from plotly.offline import plot, iplot, init_notebook_mode,plot_mpl
from plotly.offline import init_notebook_mode, plot_mpl
init_notebook_mode(connected=True)

### Load the data

In [2]:
calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
sells_price_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
sales_train_validation_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
sales_train_evalution_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
submission_df = pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv')



In [3]:
# CSS color: for plotly graphics

#                 aliceblue, antiquewhite, aqua, aquamarine, azure,
#                 beige, bisque, black, blanchedalmond, blue,
#                 blueviolet, brown, burlywood, cadetblue,
#                 chartreuse, chocolate, coral, cornflowerblue,
#                 cornsilk, crimson, cyan, darkblue, darkcyan,
#                 darkgoldenrod, darkgray, darkgrey, darkgreen,
#                 darkkhaki, darkmagenta, darkolivegreen, darkorange,
#                 darkorchid, darkred, darksalmon, darkseagreen,
#                 darkslateblue, darkslategray, darkslategrey,
#                 darkturquoise, darkviolet, deeppink, deepskyblue,
#                 dimgray, dimgrey, dodgerblue, firebrick,
#                 floralwhite, forestgreen, fuchsia, gainsboro,
#                 ghostwhite, gold, goldenrod, gray, grey, green,
#                 greenyellow, honeydew, hotpink, indianred, indigo,
#                 ivory, khaki, lavender, lavenderblush, lawngreen,
#                 lemonchiffon, lightblue, lightcoral, lightcyan,
#                 lightgoldenrodyellow, lightgray, lightgrey,
#                 lightgreen, lightpink, lightsalmon, lightseagreen,
#                 lightskyblue, lightslategray, lightslategrey,
#                 lightsteelblue, lightyellow, lime, limegreen,
#                 linen, magenta, maroon, mediumaquamarine,
#                 mediumblue, mediumorchid, mediumpurple,
#                 mediumseagreen, mediumslateblue, mediumspringgreen,
#                 mediumturquoise, mediumvioletred, midnightblue,
#                 mintcream, mistyrose, moccasin, navajowhite, navy,
#                 oldlace, olive, olivedrab, orange, orangered,
#                 orchid, palegoldenrod, palegreen, paleturquoise,
#                 palevioletred, papayawhip, peachpuff, peru, pink,
#                 plum, powderblue, purple, red, rosybrown,
#                 royalblue, saddlebrown, salmon, sandybrown,
#                 seagreen, seashell, sienna, silver, skyblue,
#                 slateblue, slategray, slategrey, snow, springgreen,
#                 steelblue, tan, teal, thistle, tomato, turquoise,
#                 violet, wheat, white, whitesmoke, yellow,
#                 yellowgreen

# Modeling

We will explore following models:

1. Naive Approach
2. Moving Average
3. Holt Linear
4. Exponential Smoothing
5. ARIMA
6. Facebook's Prophet



## Train Test Split


* We need to split sales train data into validation and train split, we will use last 50 days data as validation data, and 100 days before that as a train data. 
* We will build model using train data and test it on validaton data.



In [4]:
sales_train_validation_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3
30489,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_colums = [ col for col in sales_train_validation_df if 'd_' in col]
train_data = sales_train_validation_df[data_colums[-100:-30]]
validation_data = sales_train_validation_df[data_colums[-30:]]


### Plotting Training and Validation data

In [6]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[100].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[100].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="Training Data[Blue] Validation Data[Orange]")
fig.show()

![TrainTestSplit](https://i.imgur.com/9jDikEk.png)

## Naive Approach

Naive Approach is very simple approach, we put hypothesis that next day's sales is same as today's sale.

$\hat{Y_{t+1}} = Y_t$

here $\hat{Y_{t+1}}$  is next day's sale and $Y_t$ is today' sale. 

In [7]:
predictions = []

for index in range(len(validation_data.columns)):
    if index == 0:
        # for first validation record prediction will be last day of traindata
        predictions.append(train_data[train_data.columns[-1]].values)
    else:
        predictions.append(validation_data[validation_data.columns[index-1]].values)

# return 30490 * 30 days shape 
# for each store department 30 days prediction in same row
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
# get error for top 3 store and department record
error_naive = np.linalg.norm(predictions[:3]- validation_data.values[:3])/ len(predictions[0])
error_naive

0.30368111930480995

In [8]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[0], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[1], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[100].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[100].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[100], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="Naive Approach [Blue] Validation Data[Orange], Predictions[Green]")
fig.show()

![Naive Approach](https://i.imgur.com/3Nw4WY7.png)

### Observation:

Naive approch is not useful, it is affected by short term fluctions.

## Moving Average

* In Moving Average Model we will take the mean unit sales value from last p days, to preidct future values. p is hyper parameter which we need to decide.
* It takes into account the last p days sales, so not easily affected by short term fluctions if p is reasonably good and large.

<font size=4> $\hat{y_{t+1}} =\frac{ \sum_{t-p}^{t} y_t} {p} $ </font>

* In the above equation $y_{t+1}$ is tomorrow' sale, which can be obtained by summing over last p days' sales and take mean over this. Let's plot graph of Training Data (blue), validation data (orange), and prediction data (green).


In [9]:
len(train_data.columns)

70

In [10]:
predictions = []
for i in range(len(validation_data.columns)):
    if i == 0:
        predictions.append(np.mean(train_data[train_data.columns[-30:]].values, axis=1))
    if i < 31 and i > 0:
        # take mean of 
        train_part =np.mean(train_data[train_data.columns[-30+i:]].values, axis=1)
        validation_part = np.mean(predictions[:i], axis=0)
        predictions.append(0.5 * ( train_part + validation_part  ))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:3] - validation_data.values[:3])/len(predictions[0])


In [11]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[0], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[1], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[100].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[100].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[100], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="[Moving Average] Training Data[Blue] Validation Data[Orange], Predictions[Green]")
fig.show()

![Moving Average](https://i.imgur.com/v0Dijjq.png)

### Observation:

* Lower error compare to the naive approach, not susceptible to  day-to-day volatility, slightly able to detect trend, but still not accurace to deal with higher trends in sales. 

# Exponential Smoothing

In **Exponential smoothing** previous time steps are weighted exponentially and we add **exponentially weighted sum of previous time steps to generate predictions.** 


<font size=4> $\hat{y_{t+1}} = \alpha * y_t + \alpha* (1-\alpha) * y_{t-1} + \alpha * (1-\alpha)^2 * y_{t-2}    $ 

</font>

<font size=4> $\hat{y_{t+1}} = \alpha * y_t +  (1-\alpha) * \hat{y_{t}}    $ 

</font>





 $\alpha$   is the smoothing parameter. The forecast $y_{t+1}$ is a weighted average of all the observations in the series y1, … ,yt. The rate at which the weights decay is controlled by the parameter $\alpha$. 
 
 This method is different from simple moving average, because **weight is exponentially decay with time, so past observation have little impact on current prediction**. In simple moving average we weight every past observation equally.
 

In [12]:
train_data.columns

Index(['d_1814', 'd_1815', 'd_1816', 'd_1817', 'd_1818', 'd_1819', 'd_1820',
       'd_1821', 'd_1822', 'd_1823', 'd_1824', 'd_1825', 'd_1826', 'd_1827',
       'd_1828', 'd_1829', 'd_1830', 'd_1831', 'd_1832', 'd_1833', 'd_1834',
       'd_1835', 'd_1836', 'd_1837', 'd_1838', 'd_1839', 'd_1840', 'd_1841',
       'd_1842', 'd_1843', 'd_1844', 'd_1845', 'd_1846', 'd_1847', 'd_1848',
       'd_1849', 'd_1850', 'd_1851', 'd_1852', 'd_1853', 'd_1854', 'd_1855',
       'd_1856', 'd_1857', 'd_1858', 'd_1859', 'd_1860', 'd_1861', 'd_1862',
       'd_1863', 'd_1864', 'd_1865', 'd_1866', 'd_1867', 'd_1868', 'd_1869',
       'd_1870', 'd_1871', 'd_1872', 'd_1873', 'd_1874', 'd_1875', 'd_1876',
       'd_1877', 'd_1878', 'd_1879', 'd_1880', 'd_1881', 'd_1882', 'd_1883'],
      dtype='object')

In [13]:
train_data[train_data.columns[-30:]].values[:3].shape

(3, 30)

In [14]:

predictions = []
for row in tqdm(train_data[train_data.columns[-25:]].values[:3]):
    fit = ExponentialSmoothing(row, seasonal_periods=4).fit()
    #https://www.statsmodels.org/dev/examples/notebooks/generated/exponential_smoothing.html
    predictions.append(fit.forecast(30))
print(len(predictions[0]))
predictions = np.array(predictions).reshape((-1, 30))
print(predictions.shape)
error_exponential = np.linalg.norm(predictions[:3] - validation_data.values[:3])/len(predictions[0])
error_exponential

  0%|          | 0/3 [00:00<?, ?it/s]

30
(3, 30)


0.24177124129505864

In [15]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[0], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[1], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[2], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="[Exponential Average] Training Data[Blue] Validation Data[Orange], Predictions[Green]")
fig.show()

![Exponential Smoothing](https://i.imgur.com/WVAONGr.png)

* Exponential smoothing generating horizontal line every time,because it gives very low weightage to farwawy timesteps, causing the predictions to flatten out or remain constant.


## ARIMA



**ARIMA** stands for **A**uto **R**egressive **I**ntegrated **M**oving **A**verage.

Moving average and. exponential smoothing models were based on a description of trend and seasonality in data, ARIMA models aim to describe the correlations in the time series.

In [16]:
predictions = []
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html
# seasonal_orderiterable, optional
# The (P,D,Q,s) order of the seasonal component of the model for the AR parameters,
# differences, MA parameters, and periodicity. 
# D must be an integer indicating the integration order of the process, 
# while P and Q may either be an integers indicating the AR and MA orders 
# (so that all lags up to those orders are included) or else iterables giving specific AR and
# / or MA lags to include. s is an integer giving the periodicity (number of periods in season), 
# often it is 4 for quarterly data or 12 for monthly data. Default is no seasonal effect.
for row in tqdm(train_data[train_data.columns[-25:]].values[:3]):
    fit = sm.tsa.statespace.SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit(disp=0)
    # forecasting for 30 days
    
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
ARIMA_error = np.linalg.norm(predictions[:3] - validation_data.values[:3])/len(predictions[0])
ARIMA_error

  0%|          | 0/3 [00:00<?, ?it/s]

0.2958615466160718

In [17]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[0], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[1], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[2], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="[ARIMA] Training Data[Blue] Validation Data[Orange], Predictions[Green]")
fig.show()

![ARIMA](https://i.imgur.com/j5Y5LH8.png)

* ARIMA is able to **find low-level and high-level trends** simultaneously, unlike most other models which can only find one of these. 

* **Except the second plot**, ARIMA is able to predict a periodic function for each sample, and these functions seem to be pretty accurate. 

* Second plot most of the true value is zero and then very high value, that might be the case if  **stock of that product is not available  during that time**, which **ARIMA failed to capture, ARIMA does not have that information Available .**

## Prophet


Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

You can Learn more about [prophet](https://facebook.github.io/prophet/)

In [18]:
dates = ["2007-12-" + str(i) for i in range(1, 31)]
predictions = []
for sales in tqdm(train_data[train_data.columns[-30:]].values[:3]):
    sales_record_df = pd.DataFrame(np.transpose([dates, sales]))
    sales_record_df.columns = ["ds", "y"]
    prophet_model = Prophet(daily_seasonality=True)
    prophet_model.fit(sales_record_df)
    # prediction for next 30 days
    future = prophet_model.make_future_dataframe(periods=30)
    forecast = prophet_model.predict(future)["yhat"].loc[30:].values
    predictions.append(forecast)
    
predictions = np.array(predictions).reshape((-1, 30))
error_prophet = np.linalg.norm(predictions[:3] - validation_data.values[:3])/len(predictions[0])


  0%|          | 0/3 [00:00<?, ?it/s]


Initial log joint probability = -28.4464
Iteration  1. Log joint probability =    17.6199. Improved by 46.0663.
Iteration  2. Log joint probability =    27.5602. Improved by 9.9403.
Iteration  3. Log joint probability =    27.7191. Improved by 0.15893.
Iteration  4. Log joint probability =    28.0294. Improved by 0.31024.
Iteration  5. Log joint probability =    28.0514. Improved by 0.0219977.
Iteration  6. Log joint probability =    28.0584. Improved by 0.00697792.
Iteration  7. Log joint probability =     28.064. Improved by 0.00564507.
Iteration  8. Log joint probability =     28.066. Improved by 0.00195651.
Iteration  9. Log joint probability =    28.0666. Improved by 0.000671037.
Iteration 10. Log joint probability =    28.0678. Improved by 0.00118667.
Iteration 11. Log joint probability =    28.0684. Improved by 0.000611696.
Iteration 12. Log joint probability =    28.0685. Improved by 7.06703e-05.
Iteration 13. Log joint probability =    28.0686. Improved by 7.27821e-05.
Iterat

In [19]:

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[0], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[1], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_data.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Training Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=validation_data.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Validation Data"),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=predictions[2], mode='lines', marker=dict(color="darkgreen"), showlegend=False,
               name="Prediction Data"),
    row=3, col=1
)
fig.update_layout(height=1200, width=800, title_text="[Prophet] Training Data[Blue] Validation Data[Orange], Predictions[Green]")
fig.show()

![Prophet](https://i.imgur.com/Rc0Ivu0.png)

### Observation

Performance of Prophet is very similar to ARIMA.


## Results

In [20]:
error = [error_naive,error_avg,error_exponential,ARIMA_error, error_prophet]

names = ["Naive approach", "Moving average", "Exponential smoothing", "ARIMA", "Prophet"]

model_records = {
                "Naive approach":error_naive, "Moving average":error_avg,
                "Exponential smoothing":error_exponential, "ARIMA":ARIMA_error,
                "Prophet":error_prophet
                }
model_records = {key:val for key,val in sorted(model_records.items(), key= lambda x: x[1])}

df= pd.DataFrame.from_dict(model_records,orient='index')
df['Model'] =df.index
df.columns=["Loss", "Model"]
df.head()

,Loss,Model
Exponential smoothing,0.241771,Exponential smoothing
Moving average,0.248707,Moving average
ARIMA,0.295862,ARIMA
Prophet,0.301499,Prophet
Naive approach,0.303681,Naive approach


In [21]:
px.bar(df, y="Loss", x="Model", color="Model", title="Loss vs. Model")

![Model Results](https://i.imgur.com/aE3DrGZ.png)

### Observation and takeaways

* Exponential Smoothing has least error, Prophet seems to high.
* When we observe the prophet prediction graph, it clearly capture high and low level trend better than Naive approach, then **Why Naive approach outperformed prophet**?
* **Answer is that If we carefully look into graph, we have zero unit sales for many days, which might be the case because of zero supply, zero demand, out of stock,** which is not captured by prophet.
* We need better model which takes into **account about actual demand, supply and stocks, sometimes, holidays, price and lots of features**
* We need to **spend more time on feature engineering.**